# TODOS

1. find good config for GRPO
2. Add LoRA layer to model
3. Add PPO trainer

# Reward model training

In [1]:
import torch
import warnings
warnings.filterwarnings('ignore')
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, AutoModelForCausalLM
from trl import PPOTrainer, PPOConfig, create_reference_model
from trl import GRPOTrainer, GRPOConfig
from datasets import DatasetDict

from trl import RewardTrainer, RewardConfig

from peft import LoraConfig, get_peft_model, PeftModel

from tqdm import tqdm

import os
from dotenv import load_dotenv
from utils import CustomRewardFunction
import pandas as pd
# import sys

# # Add the parent directory to the Python path
# sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

# Load environment variables from .env file
load_dotenv()

# load the relevant devices available on the server
os.environ["CUDA_VISIBLE_DEVICES"] = os.getenv("AVAILABLE_DEVICES")

# Enable expandable CUDA segments
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# load cuda
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print("CUDA is available. Using GPU:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

There are 1 GPU(s) available.
CUDA is available. Using GPU: NVIDIA L40S


In [ ]:
MODEL = os.getenv("GENERATION_MODEL_NAME")
REWARD_MODEL = os.getenv("REWARD_MODEL_NAME")
REWARD_MODEL_EXTRACTION_LORA = os.getenv("REWARD_MODEL_EXTRACTION_LORA")
REWARD_MODEL_DETECTION_LORA = os.getenv("REWARD_MODEL_DETECTION_LORA")
RL_TOKENIZATION = "best_window"
MAX_LENGTH = int(os.getenv("RL_MAX_LENGTH"))
STRIDE = int(os.getenv("RL_STRIDE"))
PROMPT_DATASET = os.getenv("PROMPT_DATASET_CSV")
DETECTION_DIFFERENCE = int(os.getenv("DETECTION_DIFFERENCE"))
WEIGHT_EXTRACTION = float(os.getenv("WEIGHT_EXTRACTION"))
WEIGHT_DETECTION = float(os.getenv("WEIGHT_DETECTION"))
RL_TRAINING_FILES = os.getnev("RL_TRAINING_FILES")

# RL Training loop

## Load prompt dataset

In [3]:
prompt_df = pd.read_csv(PROMPT_DATASET, sep=";")
dataset = Dataset.from_pandas(prompt_df)

#TODO: test whether everything is well-separated
train_test_split = dataset.train_test_split(test_size=0.3, seed=42)
eval_test_split = train_test_split["test"].train_test_split(test_size=0.5, seed=42)


final_splits = DatasetDict({
    'train': train_test_split['train'],
    'validation': eval_test_split['train'],
    'test': eval_test_split['test']
})

In [4]:
print(prompt_df.columns)

Index(['prompt', 'precondition_texts', 'precondition_positions'], dtype='object')


In [5]:
#TODO: do train test eval split

## Load the model

### Dataset columns

1. prompt
2. precondition_text_dict --> key: id, value: text 
3. precondition_position_dict --> key: id, value: position

Think about whether any other components are needed...
Need to iterate through all preconditions to get reward, or return several rewards per response and finetune model on each one --> iteration should be good enough, just need to define reward function properly for this...

In [ ]:
# Load base model and tokenizer
model = AutoModelForCausalLM.from_pretrained(MODEL,  
                                             device_map="auto",  # For GPU/TPU acceleration
                                             torch_dtype="auto")   # Optimize precision)
tokenizer = AutoTokenizer.from_pretrained(MODEL)

# Load reward model feedback extraction
base_model = AutoModelForSequenceClassification.from_pretrained(REWARD_MODEL, num_labels=1)
reward_tokenizer = AutoTokenizer.from_pretrained(REWARD_MODEL)

extraction_model = PeftModel.from_pretrained(base_model, REWARD_MODEL_EXTRACTION_LORA).to(device)
# extraction_model = extraction_model.merge_and_unload()

detection_model = PeftModel.from_pretrained(base_model, REWARD_MODEL_DETECTION_LORA).to(device)
# detection_model = detection_model.merge_and_unload()


# Create the custom reward function
reward_function = CustomRewardFunction(extraction_model, detection_model, reward_tokenizer, MAX_LENGTH, STRIDE, RL_TOKENIZATION, device, weight_extraction=WEIGHT_EXTRACTION, weight_detection=WEIGHT_DETECTION, detection_difference=DETECTION_DIFFERENCE)

Loading checkpoint shards: 100%|██████████| 3/3 [00:02<00:00,  1.46it/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
#TODO: add lora to mistral model --> all of them...

In [ ]:
# # Define GRPO config
# grpo_config = {
#     "batch_size": 32,
#     "learning_rate": 1e-5,
#     "gamma": 0.99,  # Discount factor
#     "clip_range": 0.2,  # Similar to PPO
# }

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type='CAUSAL_LM',  # For GPT-style models
)

training_args = GRPOConfig(
    output_dir=RL_TRAINING_FILES, 
    logging_steps=10, 
    gradient_checkpointing=True,
    learning_rate=1e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    warmup_steps=100,
    logging_dir="logs",
    save_steps=1000,
    save_total_limit=2,
    eval_strategy="steps",
    eval_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    )

# Initialize GRPO trainer
trainer = GRPOTrainer(
    model=model,
    reward_funcs=reward_function,
    train_dataset=final_splits['train'],
    eval_dataset=final_splits['validation'],
    args=training_args,
    # **grpo_config
    peft_config=lora_config
)

# Train
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss


TrainOutput(global_step=54, training_loss=-0.0026001485961454885, metrics={'train_runtime': 757.7439, 'train_samples_per_second': 0.071, 'train_steps_per_second': 0.071, 'total_flos': 0.0, 'train_loss': -0.0026001485961454885})

## Code from Huggingface TRL

## Notes:

1. Do not use SLURM since I am in a single-node multi GPU setting and SLURM would work with scheduled training on a multi node cluster... --> use accelerate instead

2. install transformers accelerate deepspeed trl

3. DAPO paper for some hyperparameter settings, DeepSeekMath paper for hyperparametersettings is good as well

# PPO Training